<a href="https://colab.research.google.com/github/0xpind/wlm/blob/main/Banana_toons.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Single-cell Google Colab notebook - Animated Story Generator
# Fixed version with better JSON handling

%pip install -q -U "google-genai>=1.32.0" pillow imageio imageio-ffmpeg opencv-python numpy

import json, base64, io, time, re
from pathlib import Path
from datetime import datetime
from IPython.display import display, HTML, Markdown
from PIL import Image
import imageio.v2 as imageio
import numpy as np
import cv2
from google import genai
from google.colab import userdata

# Setup API
client = genai.Client(api_key=userdata.get("GOOGLE_API_KEY"))
MODEL = "gemini-2.5-flash-image-preview"  # Updated model

# General model guardrails (from the ordered plan)
GENERAL_GUARDRAILS = """
Visual consistency contract:
- Art style: soft, bright, kid-friendly
- Same color palette across all 5 parts
- Main character: identical proportions, costume, features
- White background #FFFFFF for character sprites
- No pure white on character/prop edges

Story structure:
- 5 parts with recap, title, 3-5 beats, hook
- 8 frames per part showing smooth animation
- Kid-friendly dialogue (5-10 words per frame)
"""

# Utilities
def extract_images(response):
    images = []
    if hasattr(response, 'candidates'):
        for candidate in response.candidates:
            if hasattr(candidate.content, 'parts'):
                for part in candidate.content.parts:
                    if hasattr(part, 'inline_data') and part.inline_data:
                        img_data = part.inline_data.data
                        if isinstance(img_data, str):
                            img_data = base64.b64decode(img_data)
                        images.append(Image.open(io.BytesIO(img_data)))
    return images

def parse_json_safely(text):
    """Extract and parse JSON from text, handling common issues"""
    # Try to find JSON block
    json_match = re.search(r'```json\s*(.*?)\s*```', text, re.DOTALL)
    if json_match:
        json_str = json_match.group(1)
    else:
        # Try to find JSON-like content
        json_start = text.find('{')
        if json_start >= 0:
            json_str = text[json_start:]
        else:
            json_str = text

    # Clean up common issues
    json_str = json_str.strip()
    # Remove trailing commas
    json_str = re.sub(r',\s*}', '}', json_str)
    json_str = re.sub(r',\s*]', ']', json_str)

    try:
        return json.loads(json_str)
    except json.JSONDecodeError as e:
        # Try to truncate at the error position
        if e.pos:
            json_str = json_str[:e.pos]
            # Find last complete object/array
            for i in range(len(json_str)-1, -1, -1):
                if json_str[i] in '}]':
                    try:
                        return json.loads(json_str[:i+1])
                    except:
                        continue
        raise e

def send_with_retry(chat, prompt, mode="text", retries=3):
    for i in range(retries):
        try:
            resp = chat.send_message(prompt)
            if mode == "text":
                return resp.text
            else:
                imgs = extract_images(resp)
                if imgs:
                    return imgs
                # Retry with image-only request
                resp = chat.send_message("Please provide only the image, no text.")
                imgs = extract_images(resp)
                if imgs:
                    return imgs
        except Exception as e:
            print(f"Retry {i+1}/{retries}: {str(e)}")
            if i == retries-1:
                raise e
            time.sleep(2 ** i)

def remove_white_bg(img):
    """Remove white background"""
    arr = np.array(img.convert("RGB"))
    # Create mask for white pixels
    white_mask = np.all(arr > 240, axis=2)
    # Create alpha channel
    alpha = np.where(white_mask, 0, 255).astype(np.uint8)
    # Create RGBA image
    rgba = np.dstack([arr, alpha])
    return Image.fromarray(rgba, "RGBA")

def composite_frame(bg, char_rgba, pos=(0.5, 0.7)):
    """Composite character on background"""
    result = bg.copy().convert("RGBA")
    x = int(pos[0] * result.width - char_rgba.width/2)
    y = int(pos[1] * result.height - char_rgba.height/2)
    result.paste(char_rgba, (x, y), char_rgba)
    return result.convert("RGB")

def create_video(frames, path, fps=10):
    writer = imageio.get_writer(str(path), fps=fps, codec="libx264", quality=8)
    for f in frames:
        writer.append_data(np.array(f.convert("RGB")))
    writer.close()

def display_video(path):
    b64 = base64.b64encode(open(path, "rb").read()).decode()
    display(HTML(f'<video width="512" controls><source src="data:video/mp4;base64,{b64}"></video>'))

# Simplified story generation
def generate_animated_story(user_prompt):
    """Generate 5-part animated story"""

    # Setup
    run_id = datetime.now().strftime("%Y%m%d_%H%M%S")
    base_dir = Path(f"story_{run_id}")
    base_dir.mkdir(exist_ok=True)

    chat = client.chats.create(model=MODEL)

    # Stage 1: Plan story
    display(Markdown("## 📝 Stage 1: Planning 5-Part Story"))

    planning_prompt = f"""
{GENERAL_GUARDRAILS}

Create a 5-part children's story from: "{user_prompt}"

Return ONLY a valid JSON object with this exact structure:
{{
  "story_title": "Title Here",
  "main_character": {{
    "name": "Character Name",
    "description": "Visual description",
    "key_features": ["feature1", "feature2", "feature3"]
  }},
  "parts": [
    {{
      "number": 1,
      "title": "Part Title",
      "recap": "",
      "beats": ["beat1", "beat2", "beat3"],
      "background": "Background description",
      "action": "What happens in 8 frames",
      "dialogue": ["Frame 1", "Frame 2", "Frame 3", "Frame 4", "Frame 5", "Frame 6", "Frame 7", "Frame 8"]
    }}
  ]
}}

Include all 5 parts. Make it educational and safe for children.
"""

    plan_text = send_with_retry(chat, planning_prompt, "text")

    try:
        story_plan = parse_json_safely(plan_text)
    except Exception as e:
        display(Markdown(f"⚠️ JSON parsing error: {e}"))
        # Fallback simple plan
        story_plan = {
            "story_title": "Adventure Story",
            "main_character": {
                "name": "Hero",
                "description": "A friendly character",
                "key_features": ["bright colors", "big smile", "simple design"]
            },
            "parts": [
                {
                    "number": i,
                    "title": f"Part {i}",
                    "recap": "" if i == 1 else f"After part {i-1}...",
                    "beats": ["Something happens", "Character reacts", "Progress made"],
                    "background": "A simple, colorful environment",
                    "action": "Character moves and interacts",
                    "dialogue": [f"Hello! Frame {j}" for j in range(1, 9)]
                }
                for i in range(1, 6)
            ]
        }

    display(Markdown(f"### Story: {story_plan.get('story_title', 'Adventure')}"))

    all_frames = []

    # Process each part
    for part_idx, part in enumerate(story_plan.get('parts', [])[:5], 1):
        display(Markdown(f"\n## 🎬 Part {part_idx}: {part.get('title', f'Part {part_idx}')}"))

        # Stage 2: Background
        display(Markdown("### Generating Background..."))
        bg_prompt = f"""
{GENERAL_GUARDRAILS}

Create a background image:
{part.get('background', 'A colorful, kid-friendly environment')}

Rules:
- Empty environment only (NO characters)
- Bright, cheerful colors
- Simple, uncluttered
- Square format
"""

        try:
            bg_images = send_with_retry(chat, bg_prompt, "image")
            background = bg_images[0].resize((512, 512))
        except:
            # Fallback: create simple colored background
            background = Image.new('RGB', (512, 512), (135, 206, 235))  # Sky blue

        # Stage 3: Character frames
        display(Markdown("### Generating Character Frames..."))
        char_frames = []

        character_desc = story_plan.get('main_character', {})

        for frame_num in range(1, 9):
            frame_prompt = f"""
{GENERAL_GUARDRAILS}

Character frame {frame_num}/8:
Character: {character_desc.get('name', 'Hero')}
Description: {character_desc.get('description', 'Friendly character')}
Features: {', '.join(character_desc.get('key_features', ['simple', 'colorful']))}

Action: {part.get('action', 'Character in motion')}
This is frame {frame_num} of 8.

Rules:
- White background #FFFFFF
- Character centered, full body
- Consistent size and style
- Simple, clear design
"""

            try:
                frame_imgs = send_with_retry(chat, frame_prompt, "image", retries=2)
                frame = frame_imgs[0].resize((512, 512))
            except:
                # Fallback: create simple character frame
                frame = Image.new('RGB', (512, 512), (255, 255, 255))

            char_frames.append(frame)

        # Stage 4: Composite
        display(Markdown("### Compositing..."))
        part_frames = []

        for i, char_frame in enumerate(char_frames):
            try:
                char_rgba = remove_white_bg(char_frame)
                final_frame = composite_frame(background, char_rgba)
            except:
                # Fallback: use background only
                final_frame = background.copy()

            part_frames.append(final_frame)
            all_frames.append(final_frame)

        # Show dialogue
        dialogues = part.get('dialogue', [f"Frame {i+1}" for i in range(8)])
        for i, d in enumerate(dialogues[:8]):
            display(Markdown(f"Frame {i+1}: *\"{d}\"*"))

        # Part preview
        try:
            part_video = base_dir / f"part{part_idx}.mp4"
            create_video(part_frames, part_video, fps=8)
            display_video(part_video)
        except:
            display(Markdown("(Preview video generation failed)"))

    # Stage 5: Final assembly
    display(Markdown("\n## 🎬 Final Story"))

    try:
        final_video = base_dir / "final_story.mp4"
        create_video(all_frames, final_video, fps=8)
        display_video(final_video)
    except:
        display(Markdown("(Final video generation failed)"))

    display(Markdown(f"\n✅ Story complete! Saved to: `{base_dir}`"))

    return str(base_dir)

# Run the generator
USER_PROMPT = "A curious banana builds rocket and goes to Mars, Ghibli style, usper high definition"
result = generate_animated_story(USER_PROMPT)

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/commands/install.py", line 377, in run
    requirement_set = resolver.resolve(
                      ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/resolution/resolvelib/resolver.py", line 95, in resolve
    result = self._result = resolver.resolve(
                            ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_vendor/resolvelib/resolvers.py", line 546, in resolve
    state = resolution.resolve(requirements, max_rounds=max_rounds)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

## 📝 Stage 1: Planning 5-Part Story

### Story: Barnaby's Big Banana Blast-Off!


## 🎬 Part 1: Barnaby's Big Idea

### Generating Background...

...

Frame 6: *"What a fun new friend!"*

Frame 7: *"Time to go home now!"*

Frame 8: *"See you again soon!"*


## 🎬 Final Story


✅ Story complete! Saved to: `story_20250908_050859`